In [1]:
import sys
sys.path.append('/mnt/code/app')
sys.path.append('/mnt/code/app/protoutils')
sys.path

['',
 '/root/anaconda3/lib/python36.zip',
 '/root/anaconda3/lib/python3.6',
 '/root/anaconda3/lib/python3.6/lib-dynload',
 '/root/anaconda3/lib/python3.6/site-packages',
 '/root/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/root/.ipython',
 '/mnt/code/app',
 '/mnt/code/app/protoutils']

In [4]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import pyspark.sql.functions as func
from pyspark.sql.types import StructField, StringType, StructType

from utils import log as logging
from common.log import def_log_cates, cust_log_cates, set_log_cate
from common.spark_utils.ddn_utils import get_service_ip, parse_time,\
    extract_data_to_cs, alter_col_name, filter_bro_logs
from common.spark_utils.ddn_config import LOGS_SCHEMA
from common.spark_utils.dsaggr_process import aggr_mssql, aggr_mysql,\
                                              dsaggr_process_sql, dsaggr_process_form, dsaggr_process_scannerform,\
                                              aggr_ds_servers, aggr_ds_services
from common.spark_utils.constants import *

print(SPARK_CODE_ROOT)

def _init_spark():
    #TODO: make the following IP configurable.
    conf = (SparkConf()
            .set("spark.cassandra.connection.host", "192.168.7.110, 192.168.7.111")
            .set("spark.cassandra.auth.username", "cassandra")
            .set("spark.cassandra.auth.password", "cassandra")
            .set("spark.driver.extraClassPath", "/opt/helios/spark/jars"))
    sc = SparkContext(conf=conf)
    sc.setLogLevel("ERROR")
    sc.addPyFile(SPARK_CODE_ROOT + '/spark_utils.zip')
    spark = SparkSession(sc)

def load_kafka_data():
    spark = SparkSession.builder.getOrCreate()
    kafka_ip = get_service_ip('connector')
    kafka_ip_port = None
    if kafka_ip is not None:
        kafka_ip_port = kafka_ip + ":" + KAFKA_PORT
        topics = ','.join(LOGS_SCHEMA.keys())

    data = spark \
        .read \
        .format("kafka") \
        .option("kafka.bootstrap.servers", kafka_ip_port) \
        .option("kafka.partition.assignment.strategy", "org.apache.kafka.clients.consumer.RoundRobinAssignor") \
        .option("subscribe", topics) \
        .load()
    data = data.selectExpr("CAST(key AS STRING)",
                           "CAST(value AS STRING)",
                           "CAST(topic AS STRING)",
                           "CAST(partition AS STRING)")
    return data

def output_data(data):
    res_df_dict = dict()
    new_partition_number = int(EXPECTED_QPS * 3600 / NUM_ENTRIES_PER_PARTITION) + 1
    for topic, _ in LOGS_SCHEMA.items():
        field_list = LOGS_SCHEMA[topic]
        source_schema = [StructField(field, StringType(), True)
                         for field in field_list]
        json_schema = StructType(source_schema)
        topic_df = data.filter(func.col('topic') == topic) \
                       .withColumn("value", func.from_json("value", json_schema)) \
                       .select('topic', 'value.*')
        topic_df = alter_col_name(topic_df)
        topic_df = parse_time(topic_df)
        topic_df = filter_bro_logs(topic_df, topic)
        print("topic %s has counts %d" % (topic, topic_df.count()))
        # topic_df.repartition(new_partition_number) \
        #         .write \
        #         .partitionBy('topic', 'year', 'month', 'day', 'hour') \
        #         .parquet(HDFS_DATA_PATH+'bro_logs', 'append')
        res_df_dict[topic] = topic_df
    return res_df_dict

def save_to_cs(res_df_dict):
    for topic, topic_df in res_df_dict.items():
        if topic_df is None \
            or topic_df.count() == 0:
            continue
        extract_data_to_cs(topic_df, topic)

def entry():
    logging.setup(logfile="/mnt/log/spark_kafka_listener_app.log")
    set_log_cate(def_log_cates | cust_log_cates)
    _init_spark()
    data = load_kafka_data()
    output_df_dict = output_data(data)
    save_to_cs(output_df_dict)

    #Here we start to put dsaggr data into cs
    mssql_join_df = aggr_mssql(output_df_dict)
    dsaggr_process_sql(mssql_join_df)
    mysql_join_df = aggr_mysql(output_df_dict)
    dsaggr_process_sql(mysql_join_df)
    dsaggr_process_form(output_df_dict, 'http')
    dsaggr_process_form(output_df_dict, 'smb_files')
    dsaggr_process_scannerform(output_df_dict)
    aggr_ds_servers()
    aggr_ds_services()

/mnt/code/target


In [3]:
entry()

Py4JJavaError: An error occurred while calling o33.addFile.
: java.io.FileNotFoundException: File file:/mnt/code/target/spark_utils.zip does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:611)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:824)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:421)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1529)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.j